# 📊 DriftWatch + MLflow — Drift Tracking Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/VincentCotella/DriftWatch/blob/main/examples/notebooks/mlflow_tracking.ipynb)

This notebook demonstrates how to **log drift detection results to MLflow** for experiment tracking.

You'll learn:

1. **Basic logging** — Log a drift report to MLflow in 3 lines
2. **Exploring results** — View metrics, params, and artifacts in the MLflow UI
3. **Multiple checks** — Track drift over time across multiple runs
4. **Pipeline integration** — Log drift alongside your training metrics

> **Requires:** `pip install driftwatch[mlflow]`

## 📦 Installation

In [ ]:
# Install DriftWatch with MLflow support
!pip install -q driftwatch[mlflow]

## 1️⃣ Setup — Generate Sample Data

We'll simulate reference (training) and production data with known drift patterns.

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)

# Reference data (Training distribution)
reference_df = pd.DataFrame(
    {
        "age": np.random.normal(30, 5, 1000),
        "income": np.random.normal(50000, 10000, 1000),
        "credit_score": np.random.normal(700, 50, 1000),
    }
)

# Production data WITH DRIFT on "age" only
production_df = pd.DataFrame(
    {
        "age": np.random.normal(45, 5, 1000),  # DRIFT: Mean 30 -> 45
        "income": np.random.normal(50000, 10000, 1000),  # No drift
        "credit_score": np.random.normal(700, 50, 1000),  # No drift
    }
)

print("Reference Data:")
print(reference_df.describe().round(2))
print("\nProduction Data:")
print(production_df.describe().round(2))

## 2️⃣ Detect Drift

Standard DriftWatch workflow — create a `Monitor` and check for drift.

In [ ]:
from driftwatch import Monitor

monitor = Monitor(
    reference_data=reference_df,
    features=["age", "income", "credit_score"],
)

report = monitor.check(production_df)

print(report.summary())

## 3️⃣ Log to MLflow — Basic Usage

Now the exciting part: **log the drift report to MLflow** with just 3 lines!

MLflow will automatically create a local `mlruns/` directory to store everything.

In [ ]:
from driftwatch.integrations.mlflow import MLflowDriftTracker

# Create a tracker pointing to an MLflow experiment
tracker = MLflowDriftTracker(
    experiment_name="drift-monitoring-demo",
)

# Log the report -> creates an MLflow run
run_id = tracker.log_report(report)

print("Logged to MLflow!")
print(f"   Run ID: {run_id}")
print(f"   Experiment: {tracker.experiment_name}")

## 4️⃣ Inspect What Was Logged

Let's use the MLflow Python client to see exactly what was recorded.

In [ ]:
import mlflow

# Fetch the run we just created
run = mlflow.get_run(run_id)

print("TAGS")
print("-" * 40)
for key, value in sorted(run.data.tags.items()):
    if key.startswith("driftwatch"):
        print(f"  {key}: {value}")

print("\nPARAMS")
print("-" * 40)
for key, value in sorted(run.data.params.items()):
    print(f"  {key}: {value}")

print("\nMETRICS")
print("-" * 40)
for key, value in sorted(run.data.metrics.items()):
    print(f"  {key}: {value}")

### View the JSON Artifact

The full drift report is also saved as a JSON artifact.

In [ ]:
import json
from pathlib import Path

# Download the artifact
artifact_path = mlflow.artifacts.download_artifacts(
    run_id=run_id, artifact_path="driftwatch/drift_report.json"
)

report_json = json.loads(Path(artifact_path).read_text())
print(json.dumps(report_json, indent=2))

## 5️⃣ Track Drift Over Time

In production, you'd run drift checks periodically (daily, hourly, etc.).

Let's simulate **3 days** of drift checks where drift gradually increases.

In [ ]:
# Simulate 3 production batches with increasing drift
drift_scenarios = [
    {"label": "Day 1 - Slight drift", "age_mean": 32, "age_std": 5},
    {"label": "Day 2 - Moderate drift", "age_mean": 38, "age_std": 6},
    {"label": "Day 3 - Severe drift", "age_mean": 50, "age_std": 8},
]

tracker = MLflowDriftTracker(
    experiment_name="drift-over-time",
    tags={"env": "production", "model": "credit-risk-v2"},
)

for scenario in drift_scenarios:
    # Generate production data for this "day"
    prod = pd.DataFrame(
        {
            "age": np.random.normal(scenario["age_mean"], scenario["age_std"], 500),
            "income": np.random.normal(50000, 10000, 500),
            "credit_score": np.random.normal(700, 50, 500),
        }
    )

    # Detect drift
    report = monitor.check(prod)

    # Log to MLflow with a descriptive run name
    run_id = tracker.log_report(
        report,
        run_name=scenario["label"],
    )

    print(
        f"{scenario['label']}: "
        f"status={report.status.value}, "
        f"drift_ratio={report.drift_ratio():.0%}, "
        f"run_id={run_id[:8]}..."
    )

print("\nAll 3 runs logged to experiment 'drift-over-time'")

### Compare Runs

Use `mlflow.search_runs()` to compare drift metrics across all runs.

In [ ]:
# Query all runs from the experiment
experiment = mlflow.get_experiment_by_name("drift-over-time")
runs_df = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["start_time ASC"],
)

# Show key metrics side by side
comparison = runs_df[
    [
        "run_id",
        "tags.mlflow.runName",
        "params.drift.status",
        "metrics.drift.drift_ratio",
        "metrics.drift.age.score",
        "metrics.drift.num_drifted",
    ]
].rename(
    columns={
        "tags.mlflow.runName": "Run Name",
        "params.drift.status": "Status",
        "metrics.drift.drift_ratio": "Drift Ratio",
        "metrics.drift.age.score": "Age Score",
        "metrics.drift.num_drifted": "Drifted Features",
    }
)

print("Drift Evolution Over Time")
print("=" * 60)
print(comparison.to_string(index=False))

## 6️⃣ Pipeline Integration

In a real ML pipeline, you often want to log drift metrics **inside the same run** as your training metrics.

Use `run_id` to attach drift data to an existing run.

In [ ]:
import mlflow

# Simulate a training pipeline
mlflow.set_experiment("training-pipeline")

with mlflow.start_run(run_name="model-training-v3") as run:
    # ---- Step 1: Train model (simulated) ----
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_metric("accuracy", 0.92)
    mlflow.log_metric("f1_score", 0.89)
    print("Model trained (accuracy=0.92, f1=0.89)")

    # ---- Step 2: Check drift on new data ----
    report = monitor.check(production_df)
    print(f"Drift check: status={report.status.value}")

    # ---- Step 3: Log drift INTO the same run ----
    tracker = MLflowDriftTracker(experiment_name="training-pipeline")
    tracker.log_report(report, run_id=run.info.run_id)
    print(f"Drift logged to same run: {run.info.run_id[:8]}...")

# Now this single run contains BOTH training metrics AND drift metrics!
final_run = mlflow.get_run(run.info.run_id)
print("\nAll metrics in this run:")
for key, value in sorted(final_run.data.metrics.items()):
    print(f"  {key}: {value}")

## 7️⃣ Custom Configuration

### Custom Metric Prefix

Use a custom prefix to namespace metrics — useful when tracking multiple models.

In [ ]:
# Track drift for two different models in the same experiment
tracker_v1 = MLflowDriftTracker(
    experiment_name="multi-model-drift",
    prefix="model_v1",  # Metrics: model_v1.has_drift, model_v1.age.score, etc.
)

tracker_v2 = MLflowDriftTracker(
    experiment_name="multi-model-drift",
    prefix="model_v2",  # Metrics: model_v2.has_drift, model_v2.age.score, etc.
)

run_id_v1 = tracker_v1.log_report(report, run_name="v1-check")
run_id_v2 = tracker_v2.log_report(report, run_name="v2-check")

print(f"Model v1 logged (prefix=model_v1): {run_id_v1[:8]}...")
print(f"Model v2 logged (prefix=model_v2): {run_id_v2[:8]}...")

### Disable Artifact Logging

If you only want metrics (no JSON file), disable artifact logging to save storage.

In [ ]:
tracker_light = MLflowDriftTracker(
    experiment_name="lightweight-tracking",
    log_report_artifact=False,  # No JSON artifact
)

run_id = tracker_light.log_report(report)
print(f"Metrics only (no artifact): {run_id[:8]}...")

## Launch the MLflow UI

To visualize all of this in the MLflow dashboard, run this in your terminal:

```bash
mlflow ui --port 5000
```

Then open [http://localhost:5000](http://localhost:5000) in your browser.

You'll see:
- Drift metrics plotted over time
- Per-feature scores for each run
- Parameters showing reference/production sizes
- Artifacts with the full JSON report

---

## Cleanup

Remove the local `mlruns/` directory created by this demo.

In [ ]:
import shutil
from pathlib import Path

mlruns_path = Path("mlruns")
if mlruns_path.exists():
    shutil.rmtree(mlruns_path)
    print("Cleaned up mlruns/ directory")
else:
    print("No mlruns/ directory to clean up")

## Summary

In this tutorial, you learned how to:

| Feature | Code |
|---------|------|
| **Basic logging** | `tracker.log_report(report)` |
| **Named runs** | `tracker.log_report(report, run_name="day-1")` |
| **Existing run** | `tracker.log_report(report, run_id=run_id)` |
| **Custom prefix** | `MLflowDriftTracker(prefix="model_v2")` |
| **Disable artifacts** | `MLflowDriftTracker(log_report_artifact=False)` |
| **Custom tags** | `MLflowDriftTracker(tags={"env": "prod"})` |

### Next Steps

- [Full Documentation](https://vincentcotella.github.io/DriftWatch/)
- [Drift Detection Tutorial](./drift_detection_tutorial.ipynb)
- [Slack Alerting](https://vincentcotella.github.io/DriftWatch/integrations/slack/)
- [FastAPI Integration](https://vincentcotella.github.io/DriftWatch/integrations/fastapi/)

---

If you found DriftWatch useful, please star us on [GitHub](https://github.com/VincentCotella/DriftWatch)!